In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scipy.stats as stats

plt.rcParams['figure.figsize'] = [5,5]
sns.set_theme(style="darkgrid")

In [ ]:

df = pd.read_csv('data.csv')

aircraft_types_key = {
'A': 'Airplane',
'B': 'Helicopter',
'C': 'Glider',
'D': 'Balloon',
'F': 'Dirigible',
'I': 'Gyroplane',
'J': 'Ultralight',
'Y': 'Other',
'Z': 'Unknown'
}
# drop rows where INCIDENT_YEAR < 2012

df = df[df['INCIDENT_YEAR'] >= 2012]

print(len(df))

col_mapping = pd.read_csv('column_mapping.csv')

airports_info = pd.read_csv('airports.csv')

df['AC_CLASS'] = df['AC_CLASS'].str.strip()

mass_map = {
    1.0 : "2,250 kg or less",
    2.0 : "2,251 - 5,700 kg",
    3.0 : "5,701 - 27,000 kg",
    4.0 : "27,001 - 272,000 kg",
    5.0 : "above 272,000 kg",
}

engine_position_map = {
    1.0 : "Engine mounted below the wing",
    2.0 : "Engine mounted above the wing",
    3.0 : "Engine is an integral part of the wing root",
    4.0 : "Engine is nacelle-mounted on the wing (i.e. piston or turboprop)",
    5.0 : "Engine is mounted on the aft fuselage",
    6.0 : "Engine is in the empennage (helicopters)",
    7.0 : "Engine mounted at the intake of the nose",
}

type_eng_map = {
    'A' : "Reciprocating engine (piston)",
    'B' : "Turbojet",
    'C' : "Turboprop",
    'D' : "Turbofan",
    'E' : "None (glider)",
    'F' : "Turboshaft (helicopter)",
    'Y' : "Other",
}

df.TYPE_ENG = df.TYPE_ENG.map(type_eng_map)


class_map = {
    'A' : 'Airplane',
    'B' : 'Helicopter',
    'C' : 'Glider',
    'D' : 'Balloon',
    'F' : 'Dirigible',
    'I' : 'Gyroplane',
    'J' : 'Ultralight',
    'Y' : 'Other',
    'Z' : 'Unknown',
}

df.AC_CLASS = df.AC_CLASS.map(class_map)

irrelevant_cols = [
    'INDEX_NR',
    'INCIDENT_DATE',
    'OPID', # operator id
    'REG',
    'AMA',
    'AMO',
    'EMA', # engine make
    'EMO', # engine model
    'COST_REPAIRS_INFL_ADJ',
    'COST_OTHER_INFL_ADJ',
    'EFFECT_OTHER',
    'OTHER_SPECIFY',
    'EFFECT_OTHER',
    'BIRD_BAND_NUMBER',
    'REMARKS',
    'REMAINS_COLLECTED',
    'REMAINS_SENT',
    'COMMENTS',
    'REPORTED_NAME',
    'REPORTED_TITLE',
    'SOURCE',
    'PERSON',
    'LUPDATE',
    'TRANSFER',
    'RUNWAY',
]

# df['IS_SPRING'] = df['INCIDENT_MONTH'].apply(lambda x: 1 if x in [3, 4, 5] else 0)
# df['IS_SUMMER'] = df['INCIDENT_MONTH'].apply(lambda x: 1 if x in [6, 7, 8] else 0)
# df['IS_FALL'] = df['INCIDENT_MONTH'].apply(lambda x: 1 if x in [9, 10, 11] else 0)
# df['IS_WINTER'] = df['INCIDENT_MONTH'].apply(lambda x: 1 if x in [12, 1, 2] else 0)
df['TIME_OF_YEAR'] = df['INCIDENT_MONTH'].apply(lambda x: 'Spring' if x in [3, 4, 5] else ('Summer' if x in [6, 7, 8] else ('Fall' if x in [9, 10, 11] else 'Winter')))

df.AC_MASS = df.AC_MASS.map(mass_map)

df.ENG_1_POS = df.ENG_1_POS.map(engine_position_map)
df.ENG_2_POS = df.ENG_2_POS.map(engine_position_map)
df.ENG_3_POS = df.ENG_3_POS.map(engine_position_map)
df.ENG_4_POS = df.ENG_4_POS.map(engine_position_map)

# df = df.drop(columns=irrelevant_cols)


all_map = {
    'STR_RAD' : 'Struck Radome',
    'DAM_RAD' : 'Damaged Radome',
    'STR_WINDSHLD' : 'Struck Windshield',
    'DAM_WINDSHLD' : 'Damaged Windshield',
    'STR_NOSE' : 'Struck Nose',
    'DAM_NOSE' : 'Damaged Nose',
    'STR_ENG1' : 'Struck Engine 1',
    'DAM_ENG1' : 'Damaged Engine 1',
    'ING_ENG1' : 'Ingested Engine 1',
    'STR_ENG2' : 'Struck Engine 2',
    'DAM_ENG2' : 'Damaged Engine 2',
    'ING_ENG2' : 'Ingested Engine 2',
    'STR_ENG3' : 'Struck Engine 3',
    'DAM_ENG3' : 'Damaged Engine 3',
    'ING_ENG3' : 'Ingested Engine 3',
    'STR_ENG4' : 'Struck Engine 4',
    'DAM_ENG4' : 'Damaged Engine 4',
    'ING_ENG4' : 'Ingested Engine 4',
    'INGESTED_OTHER' : 'Ingested Other',
    'STR_PROP' : 'Struck Propeller',
    'DAM_PROP' : 'Damaged Propeller',
    'STR_WING_ROT' : 'Struck Wing or Rotor',
    'DAM_WING_ROT' : 'Damaged Wing or Rotor',
    'STR_FUSE' : 'Struck Fuselage',
    'DAM_FUSE' : 'Damaged Fuselage',
    'STR_LG' : 'Struck Landing Gear',
    'DAM_LG' : 'Damaged Landing Gear',
    'STR_TAIL' : 'Struck Tail',
    'DAM_TAIL' : 'Damaged Tail',
    'STR_LGHTS' : 'Struck Lights',
    'DAM_LGHTS' : 'Damaged Lights',
    'STR_OTHER' : 'Struck Other',
    'DAM_OTHER' : 'Damaged Other',
}
dam_map = {
    'DAM_RAD' : 'Damaged Radome',
    'DAM_WINDSHLD' : 'Damaged Windshield',
    'DAM_NOSE' : 'Damaged Nose',
    'DAM_ENG1' : 'Damaged Engine 1',
    'DAM_ENG2' : 'Damaged Engine 2',
    'DAM_ENG3' : 'Damaged Engine 3',
    'DAM_ENG4' : 'Damaged Engine 4',
    'DAM_PROP' : 'Damaged Propeller',
    'DAM_WING_ROT' : 'Damaged Wing or Rotor',
    'DAM_FUSE' : 'Damaged Fuselage',
    'DAM_LG' : 'Damaged Landing Gear',
    'DAM_TAIL' : 'Damaged Tail',
    'DAM_LGHTS' : 'Damaged Lights',
    'DAM_OTHER' : 'Damaged Other',
}
struck_map = {
    'STR_RAD' : 'Struck Radome',
    'STR_WINDSHLD' : 'Struck Windshield',
    'STR_NOSE' : 'Struck Nose',
    'STR_ENG1' : 'Struck Engine 1',
    'STR_ENG2' : 'Struck Engine 2',
    'STR_ENG3' : 'Struck Engine 3',
    'STR_ENG4' : 'Struck Engine 4',
    'STR_PROP' : 'Struck Propeller',
    'STR_WING_ROT' : 'Struck Wing or Rotor',
    'STR_FUSE' : 'Struck Fuselage',
    'STR_LG' : 'Struck Landing Gear',
    'STR_TAIL' : 'Struck Tail',
    'STR_LGHTS' : 'Struck Lights',
    'STR_OTHER' : 'Struck Other',
}

ingested_map = {
    'ING_ENG1' : 'Ingested Engine 1',
    'ING_ENG2' : 'Ingested Engine 2',
    'ING_ENG3' : 'Ingested Engine 3',
    'ING_ENG4' : 'Ingested Engine 4',
    'INGESTED_OTHER' : 'Ingested Other',
}


def struck_str(row):
    struck_str = ''
    for key, value in struck_map.items():
        if row[key] == 1:
            struck_str += value + ', '
    return struck_str[:-2] if struck_str != '' else 'None'

def dam_str(row):
    dam_str = ''
    for key, value in dam_map.items():
        if row[key] == 1:
            dam_str += value + ', '
    return dam_str[:-2] if dam_str != '' else 'None'

def ingested_str(row):
    ingested_str = ''
    for key, value in ingested_map.items():
        if row[key] == 1:
            ingested_str += value + ', '
    return ingested_str[:-2] if ingested_str != '' else 'None'

def all_str(row):
    all_str = ''
    for key, value in all_map.items():
        if row[key] == 1:
            all_str += value + ', '
    return all_str[:-2] if all_str != '' else 'None'

df['ALL_RESULT'] = df.apply(all_str, axis=1)
df['STRUCK_RESULT'] = df.apply(struck_str, axis=1)
df['DAM_RESULT'] = df.apply(dam_str, axis=1)
df['INGESTED_RESULT'] = df.apply(ingested_str, axis=1)
damage_map = {
    'N' : 'None',
    'M' : 'Minor',
    'M?' : 'Uncertain Level',
    'S' : 'Substantial',
    'D' : 'Destroyed',
}

df['DAMAGE'] = df['DAMAGE_LEVEL'].map(damage_map)

# drop all the columns we don't need anymore
df.drop(columns=all_map.keys(), inplace=True)

df.drop(columns=['DAMAGE_LEVEL'], inplace=True)


df.DAMAGE.value_counts()

# remove K from airport ID
df['AIRPORT_ID'] = df['AIRPORT_ID'].str[1:]





## Adding data on airport traffic

In [ ]:
# set avg_yearly INCIDENT_YEAR to datetime
df['INCIDENT_YEAR'] = pd.to_datetime(df['INCIDENT_YEAR'], format='%Y')

In [ ]:
# download from: https://www.kaggle.com/datasets/sherrytp/airline-delay-analysis

flights_2012 = pd.read_csv('./flightdata/2012.csv')
flights_2013 = pd.read_csv('./flightdata/2013.csv')
flights_2014 = pd.read_csv('./flightdata/2014.csv')
flights_2015 = pd.read_csv('./flightdata/2015.csv')
flights_2016 = pd.read_csv('./flightdata/2016.csv')
flights_2017 = pd.read_csv('./flightdata/2017.csv')
flights_2018 = pd.read_csv('./flightdata/2018.csv')
flights_2019 = pd.read_csv('./flightdata/2019.csv')


In [ ]:

flights_all = pd.concat([flights_2012, flights_2013, flights_2014, flights_2015, flights_2016, flights_2017, flights_2018, flights_2019])


In [ ]:
flights_all['YEAR'] = flights_all.FL_DATE.str[:4]
flights_all['DATE'] = flights_all['FL_DATE']
flights_all['YEAR'] = pd.to_datetime(flights_all['YEAR'], format='%Y')
flights_all['DATE'] = pd.to_datetime(flights_all['DATE'], format='%Y-%m-%d')


In [ ]:
# make date just year and month
flights_all['DATE'] = flights_all['DATE'].dt.to_period('M')
flights_all['DATE'].unique()

In [ ]:
flights_all['YEAR'] = flights_all['YEAR'].dt.year
flights_all.YEAR.unique()

In [ ]:
flights_all_dep_yearly = flights_all.groupby(['ORIGIN', 'YEAR']).size().reset_index(name='counts')
flights_all_arr_yearly = flights_all.groupby(['DEST', 'YEAR']).size().reset_index(name='counts')

flights_all_dep_monthly = flights_all.groupby(['ORIGIN', 'DATE']).size().reset_index(name='counts')
flights_all_arr_monthly = flights_all.groupby(['DEST', 'DATE']).size().reset_index(name='counts')

In [ ]:
flights_all_dep_monthly.head(10)

In [ ]:
flights_all_dep_yearly['Location'] = flights_all_dep_yearly['ORIGIN']
flights_all_arr_yearly['Location'] = flights_all_arr_yearly['DEST']
flights_all_dep_yearly['Departures'] = flights_all_dep_yearly['counts']
flights_all_arr_yearly['Arrivals'] = flights_all_arr_yearly['counts']
flights_all_dep_yearly.drop(columns=['ORIGIN', 'counts'], inplace=True)
flights_all_arr_yearly.drop(columns=['DEST', 'counts'], inplace=True)

flights_all_dep_monthly['Location'] = flights_all_dep_monthly['ORIGIN']
flights_all_arr_monthly['Location'] = flights_all_arr_monthly['DEST']
flights_all_dep_monthly['DEP_MONTHLY'] = flights_all_dep_monthly['counts']
flights_all_arr_monthly['ARR_MONTHLY'] = flights_all_arr_monthly['counts']
flights_all_dep_monthly.drop(columns=['ORIGIN', 'counts'], inplace=True)
flights_all_arr_monthly.drop(columns=['DEST', 'counts'], inplace=True)

flights_all_monthly = pd.merge(flights_all_arr_monthly, flights_all_dep_monthly, on=['Location', 'DATE'], how='outer')

flights_all_yearly = pd.merge(flights_all_arr_yearly, flights_all_dep_yearly, on=['Location', 'YEAR'], how='outer')

In [ ]:
flights_all_monthly.head()

In [ ]:
flights_all_yearly['AIRPORT_ID'] = flights_all_yearly['Location']
flights_all_yearly.drop(columns=['Location'], inplace=True)
flights_all_yearly['INCIDENT_YEAR'] = flights_all_yearly['YEAR']
flights_all_yearly.drop(columns=['YEAR'], inplace=True)
flights_all_yearly['Total'] = flights_all_yearly['Arrivals'] + flights_all_yearly['Departures']
flights_all_yearly['Total'] = flights_all_yearly['Total'].fillna(0)
flights_all_yearly['Arrivals'] = flights_all_yearly['Arrivals'].fillna(0)
flights_all_yearly['Departures'] = flights_all_yearly['Departures'].fillna(0)
flights_all_yearly['Total'] = flights_all_yearly['Total'].astype(int)
flights_all_yearly['Arrivals'] = flights_all_yearly['Arrivals'].astype(int)
flights_all_yearly['Departures'] = flights_all_yearly['Departures'].astype(int)

flights_all_monthly['AIRPORT_ID'] = flights_all_monthly['Location']
flights_all_monthly.drop(columns=['Location'], inplace=True)
flights_all_monthly['TOTAL_MONTHLY'] = flights_all_monthly['ARR_MONTHLY'] + flights_all_monthly['DEP_MONTHLY'] 
flights_all_monthly['TOTAL_MONTHLY'] = flights_all_monthly['TOTAL_MONTHLY'].fillna(0)
flights_all_monthly['ARR_MONTHLY'] = flights_all_monthly['ARR_MONTHLY'].fillna(0) 
flights_all_monthly['DEP_MONTHLY'] = flights_all_monthly['DEP_MONTHLY'].fillna(0) 
flights_all_monthly['TOTAL_MONTHLY'] = flights_all_monthly['TOTAL_MONTHLY'].astype(int)
flights_all_monthly['ARR_MONTHLY'] = flights_all_monthly['ARR_MONTHLY'].astype(int) 
flights_all_monthly['DEP_MONTHLY'] = flights_all_monthly['DEP_MONTHLY'].astype(int)


In [ ]:

# date format: Month/Day/Year
df.INCIDENT_DATE = pd.to_datetime(df.INCIDENT_DATE, format='%m/%d/%y')
df.INCIDENT_DATE.unique()

In [ ]:
df['INCIDENT_DATE_MONTHLY'] = df.INCIDENT_DATE.dt.to_period('M')

In [ ]:
df.INCIDENT_DATE_MONTHLY.unique()

In [ ]:
flights_all_monthly.DATE.unique()

In [ ]:
flights_all_yearly.INCIDENT_YEAR = pd.to_datetime(flights_all_yearly.INCIDENT_YEAR, format='%Y')
df.INCIDENT_YEAR = pd.to_datetime(df.INCIDENT_YEAR, format='%Y')
df.INCIDENT_MONTH = pd.to_datetime(df.INCIDENT_MONTH, format='%Y-%m')

In [ ]:
df_flights = pd.merge(df, flights_all_yearly, on=['AIRPORT_ID', 'INCIDENT_YEAR'], how='left')

In [ ]:
df_flights.INCIDENT_YEAR = pd.to_datetime(df_flights.INCIDENT_YEAR, format='%Y')


In [ ]:
df_flights['INCIDENT_YEAR'] = df_flights['INCIDENT_YEAR'].dt.year


In [ ]:

df_flights.INCIDENT_YEAR.unique()

In [ ]:
flights_all_monthly['INCIDENT_DATE_MONTHLY'] = flights_all_monthly['DATE']
flights_all_monthly.drop(columns=['DATE'], inplace=True)


In [ ]:
df_flights2 = pd.merge(df_flights, flights_all_monthly, on=['AIRPORT_ID', 'INCIDENT_DATE_MONTHLY'], how='left')

In [ ]:
df_flights2.INCIDENT_MONTH = df_flights2.INCIDENT_MONTH.dt.month

In [ ]:
df_flights2.head()

## Determining the type of aircraft that has the most incidents



In [ ]:
df = df_flights2

### Determining which airports have the most incidents

We can see that KDEN (Denver International Airport) has the most incidents, followed by KDFW (Dallas/Fort Worth International Airport) and KORD (Chicago O'Hare International Airport).
These locations are a good place to start our analysis.


In [ ]:
flights_all_monthly['MONTH'] = flights_all_monthly['INCIDENT_DATE_MONTHLY'].dt.month

In [ ]:
flights_all_monthly.head()

In [ ]:
# plot top 20 airports by number of incidents
df[df.AIRPORT_ID != 'ZZZZ'].AIRPORT_ID.value_counts().head(10).plot(kind='bar')
plt.xlabel('Airport ID')
plt.ylabel('Number of Incidents')
plt.title('Top 10 Airports by Number of Incidents')
plt.show()
n = 5
# top_airports = df[df.AIRPORT_ID != 'ZZZ'].AIRPORT_ID.value_counts().head(n).index.tolist()
top_airports = ['DEN', 'DFW', 'CLT', 'ORD', 'IAH']
print(f'Top {n} airports: {top_airports}')

df_airports = df[df.AIRPORT_ID.isin(top_airports)]
# avg_yearly_airports = avg_yearly[avg_yearly.AIRPORT_ID.isin(top_airports)]
avg_yearly_airports = flights_all_yearly[flights_all_yearly.AIRPORT_ID.isin(top_airports)]

avg_monthly_airports = flights_all_monthly[flights_all_monthly.AIRPORT_ID.isin(top_airports)]


df_airports.AIRPORT_ID.value_counts().plot(kind='bar')
plt.xlabel('Airport ID')
plt.ylabel('Number of Incidents')
plt.title(f'Top {n} Airports by Number of Incidents')
plt.show()


# plot top 20 species by number of incidents
# df_airports.SPECIES.value_counts().head(20).plot(kind='bar')
# plt.xlabel('Species')
# plt.ylabel('Number of Incidents')
# plt.title('Top 20 Species by Number of Incidents')
# plt.show()


In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
# top 10 airports by number of total flights
top10_airports = flights_all_yearly.groupby('AIRPORT_ID').Total.sum().sort_values(ascending=False).head(10).index.tolist()
top10_airports_flights = flights_all_yearly[flights_all_yearly.AIRPORT_ID.isin(top10_airports)]
top10_airports_flights.groupby('AIRPORT_ID').Total.sum().sort_values(ascending=False).plot(kind='bar')

In [ ]:
top10_airports_flights = top10_airports_flights.groupby('AIRPORT_ID').Total.sum().sort_values(ascending=False)

In [ ]:
top10_airports_incidents = df[df.AIRPORT_ID.isin(top10_airports)].value_counts('AIRPORT_ID')
# plto top10_airports_incidents and top10_airports_flights

In [ ]:
top10df = pd.DataFrame({'Incidents': top10_airports_incidents, 'Flights': top10_airports_flights})

top10df['Proportion'] = top10df['Incidents'] / top10df['Flights']

top10df['Proportion'].sort_values(ascending=False).plot(kind='bar')
plt.title('Proportion of Incidents to Flights')
plt.xlabel('Airport ID')
plt.ylabel('Proportion')
plt.show()

In [ ]:


flights_all_yearly.groupby('AIRPORT_ID').Total.sum().sort_values(ascending=False).head(10).plot(kind='bar')
# plot the number of incidents for each airport
plt.xlabel('Airport ID')
plt.ylabel('Number of Flights')
plt.title('Top 10 Airports by Number of Flights')
# make x ticks horizontal
plt.xticks(rotation=0)
plt.show()



In [ ]:
# top 10 airports by average yearly number of flights
flights_all_yearly.groupby('AIRPORT_ID').Total.mean().sort_values(ascending=False).head(10).plot(kind='bar')
plt.xlabel('Airport ID')
plt.ylabel('Number of Flights')
plt.title('Top 10 Airports by Average Yearly Number of Flights')
# make x ticks horizontal
plt.xticks(rotation=0)
plt.show()

In [ ]:
# year at all airports
sns.displot(df_airports, x='INCIDENT_YEAR', kind='kde', fill=True, aspect=2)
plt.xlabel('Year')
plt.ylabel('Count')
plt.title(f'Incidents by Year at All Airports')

In [ ]:
# total flights per year at all airports
plt.rcParams['figure.figsize'] = [10,5]
avg_yearly_total = avg_yearly_airports.groupby(['INCIDENT_YEAR']).agg({'Total': 'sum'}).reset_index()
sns.lineplot(data=avg_yearly_total, x='INCIDENT_YEAR', y='Total')
plt.xlabel('Year')
plt.ylabel('Total Flights')
plt.title(f'Total Flights by Year at All Airports')


In [ ]:
# total flights per year at top airports
plt.rcParams['figure.figsize'] = [10,5]
# line plot, smooth line, with error bands, aspect ratio 2
sns.lineplot(data=avg_yearly_airports, x='INCIDENT_YEAR', y='Total', hue='AIRPORT_ID') 
plt.xlabel('Year')
plt.ylabel('Total Flights')
plt.title(f'Total Flights per Year at Top {n} Airports by Incident Count')
plt.show()


In [ ]:
# year
sns.displot(df_airports, x='INCIDENT_YEAR', hue='AIRPORT_ID', kind='hist', fill=True, aspect=2)
plt.xlabel('Year')
plt.ylabel('Density')
# make xticks every year
plt.xticks(np.arange(2012, 2023, 1))
# center bars on xticks
plt.title(f'Incidents by Year at Top {n} Airports')
plt.show()

In [ ]:
avg_monthly_airports.head()

In [ ]:
# monthly flights at top airports, histogram
sns.barplot(data=avg_monthly_airports, x='MONTH', y='TOTAL_MONTHLY', hue='AIRPORT_ID')
plt.xlabel('Month')
plt.ylabel('Total Flights')
plt.title(f'Total Flights per Month at Top {n} Airports by Incident Count')
plt.show()

In [ ]:
# monthly flights at top airports
plt.rcParams['figure.figsize'] = [10,5]
# plot avg_monthly_airports, month on x axis, total flights on y axis, hue by airport
sns.lineplot(data=avg_monthly_airports, x='MONTH', y='TOTAL_MONTHLY', hue='AIRPORT_ID')
plt.xlabel('Month')
plt.ylabel('Total Flights')
plt.title(f'Total Flights per Month at Top {n} Airports')
plt.show()

In [ ]:
df_airports.columns

In [ ]:
# month
sns.displot(df_airports, x='MONTH', hue='AIRPORT_ID', kind='kde', fill=True, aspect=2)
plt.xlabel('Month')
plt.ylabel('Density')
# show xticks every month
plt.title(f'Incidents by Month at Top {n} Airports')
plt.show()

In [ ]:

# speed
sns.displot(df_airports, x='SKY', hue='AIRPORT_ID', kind='hist', fill=True, aspect=2)
plt.xlabel('Sky')
plt.ylabel('Density')
plt.title(f'Incidents by Sky at Top {n} Airports')
plt.show()

In [ ]:

# altitude
sns.displot(df_airports, x='HEIGHT', hue='AIRPORT_ID', kind='kde', fill=True, aspect=2)
plt.xlabel('Altitude (Feet)')
plt.ylabel('Density')
plt.title(f'Incidents by Altitude at Top {n} Airports')
plt.show()

In [ ]:
# PHASE_OF_FLIGHT
sns.displot(df_airports, x='PHASE_OF_FLIGHT', hue='AIRPORT_ID', kind='hist', fill=True, aspect=2)
plt.xlabel('Phase of Flight')
plt.ylabel('Density')
plt.title(f'Incidents by Phase of Flight at Top {n} Airports')
plt.show()

In [ ]:
# AC_MASS
sns.displot(df_airports, x='AC_MASS', hue='AIRPORT_ID', kind='hist', fill=True, aspect=2)
plt.xlabel('Aircraft Mass (Pounds)')
plt.ylabel('Number of Incidents')
plt.title(f'Incidents by Aircraft Mass at Top {n} Airports')
plt.show()

In [ ]:
# TYPE_ENG
sns.displot(df_airports, x='TYPE_ENG', hue='AIRPORT_ID', kind='hist', fill=True, aspect=2)
plt.xlabel('Engine Type')
plt.ylabel('Number of Incidents')
plt.title(f'Incidents by Engine Type at Top {n} Airports')
plt.show()

In [ ]:
# height and speed
sns.displot(df_airports, x='SPEED', y='HEIGHT',  kind='kde', fill=True, aspect=2)
# sns.histplot(df_airports, x='SPEED', y='HEIGHT', hue='AIRPORT_ID', bins=20, pthresh=.1, cmap='mako')
plt.xlabel('Speed (Knots)')
plt.ylabel('Altitude (Feet)')
plt.title(f'Incidents by Speed and Altitude at Top {n} Airports')
plt.show()


In [ ]:
# precipitation
sns.displot(df_airports, x='PRECIPITATION', hue='AIRPORT_ID', kind='hist', fill=True, aspect=2)
plt.xlabel('Precipitation (Inches)')
plt.ylabel('Number of Incidents')
plt.title(f'Incidents by Precipitation at Top {n} Airports')
plt.show()

# all airports
sns.displot(df, x='PRECIPITATION',kind='hist', fill=True, aspect=2)
plt.xlabel('Precipitation (Inches)')
plt.ylabel('Number of Incidents')
plt.title(f'Incidents by Precipitation at All Airports')
plt.show()

In [ ]:
# EFFECT
sns.displot(df_airports, x='EFFECT', hue='AIRPORT_ID', kind='hist', fill=True, aspect=2)
plt.xlabel('Effect')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45)
plt.title(f'Incidents by Effect at Top {n} Airports')
plt.show()

In [ ]:
# drop rows with no effect
df_airports_effected = df_airports[df_airports.EFFECT != 'None']
sns.displot(df_airports_effected, x='EFFECT', hue='AIRPORT_ID', kind='hist', fill=True, aspect=2)
plt.xlabel('Effect')
plt.ylabel('Number of Incidents')
plt.xticks(rotation=45)
plt.title(f'Incidents by Effect at Top {n} Airports')
plt.show()


In [ ]:

sns.displot(df_airports_effected, x='SKY', hue='EFFECT', kind='hist', fill=True, aspect=2)



In [ ]:
sns.displot(df_airports_effected, x='PRECIPITATION', hue='EFFECT', kind='hist', fill=True, aspect=2)


In [ ]:
sns.displot(df_airports_effected, x='EFFECT', hue='WARNED', kind='hist', fill=True, aspect=2)
plt.xticks(rotation=45)

In [ ]:
# ALL_RESULT, 10 most common
df_effected = df[df.EFFECT != 'None']

df_airports_effected_result = df_effected[df_effected.ALL_RESULT != 'None']
df_airports_effected_result = df_airports_effected_result[df_airports_effected_result.ALL_RESULT != 'Struck Other']
idx = df_airports_effected_result.ALL_RESULT.value_counts().head(10).index
sns.displot(df_airports_effected_result.loc[df_airports_effected_result.ALL_RESULT.isin(idx)], x='ALL_RESULT', hue='EFFECT', kind='hist', fill=True, aspect=2)
plt.xticks(rotation=45)
plt.xlabel('Result')
plt.ylabel('Number of Incidents')
plt.title(f'Incidents by Result at Top {n} Airports, only incidents with an effect')


In [ ]:
df_damaged = df[df.DAMAGE != 'None']

# damage level by phase of flight
sns.displot(df_damaged, x='PHASE_OF_FLIGHT', hue='DAMAGE', kind='hist', fill=True, aspect=2)
plt.xticks(rotation=45)
plt.xlabel('Phase of Flight')
plt.ylabel('Number of Incidents')
plt.title(f'Incidents by Phase of Flight at Top {n} Airports, only incidents with damage')
plt.show()

In [ ]:
df_airports.head()

# Stat Analysis 


**Tests:**
- Is the proportion of incidents that occur at the top 3 airports significantly different from the proportion of flights that occur at the top 3 airports?
  - Result: The proportion of incidents that occur at the top 3 airports is significantly different from the proportion of flights that occur at the top 3 airports at the 0.05 level.
- Is the proportion of incidents that occur during the months 5, 6, 7 at denver airport significantly different from the proportion of flights that occur during the months 5, 6, 7 at denver airport?
- Is the proportion of incidents that occur during each month at the top 3 airports singificantly different from the proportion of flights that occur during each month at the top 3 airports?

- Is the proportion of incidents that occur during Landing at Denver airport significantly different from the proportion of incidents that occur during Landing on average?

- Is the proportion of incidents that occur during Approach at DFW airport significantly different from the proportion of incidents that occur during Approach on average?
- Is the proportion of incidents that occur during Take-off at Denver airport significantly different from the proportion of incidents that occur during Take-off on average?
- Is the proportion of incidents that occur at low altitude at Denver airport significantly different from the proportion of incidents that occur at low altitude on average?
- Is the proportion of incidents that occur with aircraft between 27,0001 and 272,000 kg at Denver airport significantly different from the proportion of incidents that occur with aircraft between 27,0001 and 272,000 kg on average?

### Is the proportion of incidents that occur at the top 3 airports significantly different from the proportion of flights that occur at the top 3 airports?

In [ ]:
# Chi-Square test for homogeneity. 

# H0: The proportion of incidents that occur at the top 3 airports is the same as the proportion of flights that occur at the top 3 airports.
# H1: The proportion of incidents that occur at the top 3 airports is not the same as the proportion of flights that occur at the top 3 airports.


# top_airports: list of top 3 airports
# flights_all_yearly where AIRPORT_ID is in top_airports

flights_chi2 = flights_all_yearly[flights_all_yearly.AIRPORT_ID.isin(top_airports)]
flights_chi2 = flights_chi2.groupby('AIRPORT_ID').sum().reset_index()
flights_chi2 = flights_chi2[['AIRPORT_ID', 'Total']]
flights_chi2.columns = ['AIRPORT_ID', 'FLIGHTS']
flights_chi2.head()

# make AIRPORT_ID 'Other' for all airports not in top_airports
other_amount = flights_all_yearly[flights_all_yearly.AIRPORT_ID.isin(top_airports) == False].sum().Total

flights_chi2.loc[len(flights_chi2.index)] = ['Other', other_amount]
flights_chi2.head()


In [ ]:

# incidents_chi2: incidents at top_airports
df_airports['INCIDENTS'] = 1
incidents_chi2 = df_airports.groupby('AIRPORT_ID').sum().reset_index()
incidents_chi2 = incidents_chi2[['AIRPORT_ID', 'INCIDENTS']]
incidents_chi2.head()

other_incident_amount = df[df.AIRPORT_ID.isin(top_airports) == False].shape[0]
incidents_chi2.loc[len(incidents_chi2.index)] = ['Other', other_incident_amount]
incidents_chi2.head()


In [ ]:
# make 2 way chi-square table
incidents_chi2

incidents = incidents_chi2.INCIDENTS.to_list()
flights = flights_chi2.FLIGHTS.to_list()
obs = np.array([incidents, flights])

In [ ]:
statistic, pvalue, dof, expected_freq = stats.chi2_contingency(obs)
# statistic: chi-square statistic
# pvalue: p-value
# dof: degrees of freedom
# expected_freq: expected frequencies

print(f'Chi-Square statistic: {statistic}')
print(f'p-value: {pvalue}')
print(f'Degrees of Freedom: {dof}')
print(f'Expected Frequencies: {expected_freq}')

if pvalue < 0.05:
    print('Reject the null hypothesis at the 0.05 significance level. (pvalue < 0.05)')
    print('The proportion of incidents that occur at the top 3 airports is not the same as the proportion of flights that occur at the top 3 airports.')



### Is the proportion of incidents that occur during the months 5, 6, 7 at denver airport significantly different from the proportion of flights that occur during the months 5, 6, 7 at denver airport?

In [ ]:
# H0: The proportion of incidents that occur at the top 3 airports is the same as the proportion of flights that occur at the top 3 airports.
# H1: The proportion of incidents that occur at the top 3 airports is not the same as the proportion of flights that occur at the top 3 airports.

flights_chi2_2 = flights_all_monthly[flights_all_monthly.AIRPORT_ID.isin(top_airports)]
flights_chi2_2 = flights_chi2_2.groupby(['AIRPORT_ID', 'MONTH']).sum().reset_index()
flights_chi2_2 = flights_chi2_2[['AIRPORT_ID', 'MONTH', 'TOTAL_MONTHLY']]


In [ ]:
other_amount_monthly = flights_all_monthly[flights_all_monthly.AIRPORT_ID.isin(top_airports) == False].groupby('MONTH').sum().reset_index()

other_amount_monthly['AIRPORT_ID'] = 'Other'
other_amount_monthly = other_amount_monthly[['AIRPORT_ID', 'MONTH', 'TOTAL_MONTHLY']]

flights_chi2_2 = flights_chi2_2.append(other_amount_monthly)

In [ ]:
incidents_chi2_2 = df_airports.groupby(['AIRPORT_ID', 'MONTH']).sum().reset_index()
incidents_chi2_2 = incidents_chi2_2[['AIRPORT_ID', 'MONTH', 'INCIDENTS']]

other_incident_amount_monthly = []
for i in range(12):
    count = df[(df.AIRPORT_ID.isin(top_airports) == False) & (df.MONTH == i+1)].shape[0]
    other_incident_amount_monthly.append(count)
    

In [ ]:
other_incident_amount_monthly = pd.DataFrame({'AIRPORT_ID': ['Other']*12, 'MONTH': range(1,13), 'INCIDENTS': other_incident_amount_monthly})
other_incident_amount_monthly = incidents_chi2_2.append(other_incident_amount_monthly)

In [ ]:
other_incident_amount_monthly['IS_SUMMER'] = other_incident_amount_monthly.MONTH.isin([5,6,7]) # months May, June, July
flights_chi2_2['IS_SUMMER'] = flights_chi2_2.MONTH.isin([5,6,7]) # months May, June, July

other_incident_amount_monthly.IS_SUMMER = other_incident_amount_monthly.IS_SUMMER.astype(int)
flights_chi2_2.IS_SUMMER = flights_chi2_2.IS_SUMMER.astype(int)

In [ ]:
incidents_chi2_2 = other_incident_amount_monthly

In [ ]:
# reset index for both dataframes
flights_chi2_2 = flights_chi2_2.reset_index()
incidents_chi2_2 = incidents_chi2_2.reset_index()
flights_chi2_2 = flights_chi2_2[['AIRPORT_ID', 'MONTH', 'TOTAL_MONTHLY']]
incidents_chi2_2 = incidents_chi2_2[['AIRPORT_ID', 'MONTH', 'INCIDENTS', 'IS_SUMMER']]

In [ ]:
table2 = incidents_chi2_2.groupby(['AIRPORT_ID', 'IS_SUMMER']).sum().reset_index()
table2 = table2[['AIRPORT_ID', 'IS_SUMMER', 'INCIDENTS']]


In [ ]:
top_airports

In [ ]:
den_incidents = incidents_chi2_2[incidents_chi2_2.AIRPORT_ID == 'DEN'].INCIDENTS.to_list()
dfw_incidents = incidents_chi2_2[incidents_chi2_2.AIRPORT_ID == 'DFW'].INCIDENTS.to_list()
clt_incidents = incidents_chi2_2[incidents_chi2_2.AIRPORT_ID == 'CLT'].INCIDENTS.to_list()
ord_incidents = incidents_chi2_2[incidents_chi2_2.AIRPORT_ID == 'ORD'].INCIDENTS.to_list()
iah_incidents = incidents_chi2_2[incidents_chi2_2.AIRPORT_ID == 'IAH'].INCIDENTS.to_list()
other_incidents = incidents_chi2_2[incidents_chi2_2.AIRPORT_ID == 'Other'].INCIDENTS.to_list()

print(f'DEN incidents: {den_incidents}')
print(f'DFW incidents: {dfw_incidents}')
print(f'CLT incidents: {clt_incidents}')
print(f'ORD incidents: {ord_incidents}')
print(f'IAH incidents: {iah_incidents}')
print(f'Other incidents: {other_incidents}')



In [ ]:
statistic, pvalue, dof, expected_freq = stats.chi2_contingency([den_incidents, dfw_incidents, ord_incidents, other_incidents])
print(f'Chi-Square statistic: {statistic}')
print(f'p-value: {pvalue}')
print(f'Degrees of Freedom: {dof}')
print(f'Expected Frequencies: {expected_freq}')

if pvalue < 0.05:
    print('Reject the null hypothesis at the 0.05 significance level. (pvalue < 0.05)')
    print('The proportion of incidents that occur at the top 3 airports during May, June, and July is not the same as the proportion of incidents that occur at the top 3 airports during the other months.')

### Is the proportion of incidents that occur during May, June, July at Denver airport significantly different from the proportion of incidents that occur during May, June, July at the top 3 airports?


In [ ]:
# make df where each row is a combination of airport_id and is_summer

temp = pd.DataFrame()
for i in range(len(incidents_chi2_2)):
    airport_id = incidents_chi2_2.iloc[i].AIRPORT_ID
    if airport_id == 'Other':
        continue
    is_summer = incidents_chi2_2.iloc[i].IS_SUMMER
    incidents = incidents_chi2_2.iloc[i].INCIDENTS
    identifier = airport_id + '_SUMMER' if is_summer == 1 else airport_id + '_NOT_SUMMER' 
    arr = [identifier, incidents]
    temp = temp.append(pd.DataFrame(arr).T)


In [ ]:
# rename columns
temp.columns = ['IDENTIFIER', 'INCIDENTS']


In [ ]:
temp.head(100)

In [ ]:
temp.INCIDENTS = temp.INCIDENTS.astype(int)
temp.IDENTIFIER = temp.IDENTIFIER.astype(str)
tukey = pairwise_tukeyhsd(temp.INCIDENTS, temp.IDENTIFIER, alpha=0.05)
print(tukey)

In [ ]:
temp.head()

In [ ]:
df.PHASE_OF_FLIGHT.unique()

### Is the proportion of incidents that occur during Landing, Approach, and Take-off at the top 5 airports significantly different from the proportion of incidents that occur in other phases of flight?

In [ ]:
# H0: The proportion of incidents that occur during Landing, Approach, and Takeoff is the same as the proportion of incidents that occur during other phases of flight.
# H1: The proportion of incidents that occur during Landing, Approach, and Takeoff is not the same as the proportion of incidents that occur during other phases of flight.
# 
# alpha = 0.05
# 

incidents_phases = {'Landing Roll': 0, 'Approach': 0, 'Take-off Run': 0, 'Other': 0}
chosen = ['Landing Roll', 'Approach', 'Take-off Run']


for i in range(len(df)):
    phase = df.iloc[i].PHASE_OF_FLIGHT
    if phase in chosen:
        incidents_phases[phase] += 1
    else:
        incidents_phases['Other'] += 1



In [ ]:

vals = list(incidents_phases.values())
result = stats.chisquare(vals)


In [ ]:
tukey = pairwise_tukeyhsd(df.INCIDENT, df.PHASE_OF_FLIGHT, alpha=0.05)


In [ ]:
print(f'Chi-Square statistic: {result.statistic}')
print(f'p-value: {result.pvalue}')
if result.pvalue < 0.05:
    print('Reject the null hypothesis at the 0.05 significance level. (pvalue < 0.05)')
    print('The proportion of incidents that occur during Landing, Approach, and Takeoff is not the same as the proportion of incidents that occur during other phases of flight.')
    

### Is the proportion of incidents that occur during the "summer months" greater than the proportion of incidents that occur during other months at all airports?



In [ ]:
incidents_monthly = {'Summer' : 0, 'Other' : 0}
for i in range(len(df)):
    month = df.iloc[i].MONTH
    if month in [5,6,7]:
        incidents_monthly['Summer'] += 1
    else:
        incidents_monthly['Other'] += 1

vals = list(incidents_monthly.values())
result = stats.chisquare(vals)

print(f'Chi-Square statistic: {result.statistic}')
print(f'p-value: {result.pvalue}')
# if result.pvalue < 0.05:



In [ ]:
df.columns

### Is the proportion of incidents resulting in injury or death in small airplanes significantly different from the proportion of incidents resulting in injury or death in other aircraft types?


In [ ]:
df.NR_INJURIES.isna

In [ ]:
non_injury = df[df.NR_INJURIES.isna()]
non_death = df[df.NR_FATALITIES.isna()]
death_incidents = df[df.NR_FATALITIES > 0]
injury_incidents = df[df.NR_INJURIES > 0]

df['NON_INJURY'] = df.NR_INJURIES.isna().astype(int)
df['NON_DEATH'] = df.NR_FATALITIES.isna().astype(int)


df['INJURY'] = df.NR_INJURIES.notna().astype(int)
df['DEATH'] = df.NR_FATALITIES.notna().astype(int)

# 'Death' if DEATH == 1 else 'Injury' if INJURY == 1 else 'Non-Injury' if NON_INJURY == 1 else 'Non-Death' if NON_DEATH == 1 else 'Unknown'
df['INJURY_DEATH_STATUS'] = df.apply(lambda x: 'Death' if x.DEATH == 1 else 'Injury' if x.INJURY == 1 else 'Non-Injury' if x.NON_INJURY == 1 else 'Non-Death' if x.NON_DEATH == 1 else 'None', axis=1)




In [ ]:
# add column called 'INCIDENT' == 1 for all rows
df['INCIDENT'] = 1
df.head()

In [ ]:
df_injury = df.groupby(['AC_MASS', 'INJURY_DEATH_STATUS']).INCIDENT.sum().reset_index()


In [ ]:
df_injury.head(20)

In [ ]:
# tukey parwise hsd
# response variable: 


# make df where each row is a combination of AC_MASS and INJURY_DEATH_STATUS

temp2 = pd.DataFrame()

temp2 = pd.DataFrame()
for i in range(len(df_injury)):
    ac_mass = df_injury.iloc[i].AC_MASS
    injury_death_status = df_injury.iloc[i].INJURY_DEATH_STATUS
    incidents = df_injury.iloc[i].INCIDENT
    identifier = ac_mass + '_' + injury_death_status
    arr = [identifier, incidents]
    temp2 = temp2.append(pd.DataFrame(arr).T)

# rename columns
temp2.columns = ['IDENTIFIER', 'INCIDENTS']



In [ ]:
temp2.INCIDENTS = temp2.INCIDENTS.astype(int)
temp2.IDENTIFIER = temp2.IDENTIFIER.astype(str)
# reset index
temp2 = temp2.reset_index(drop=True)
temp2.head(100)


In [ ]:
# chi-square test
print(pairwise_tukeyhsd(temp2.INCIDENTS, temp2.IDENTIFIER, alpha=0.05))


In [ ]:

# plto the distribution of non-injury incidents
plt.figure(figsize=(5,5))
non_injury.AC_MASS.value_counts().plot(kind='bar')
plt.title('Distribution of Non-Injury Incidents')
plt.xlabel('Aircraft Mass')
plt.ylabel('Number of Incidents')
plt.show()

# plot the distribution of non-death incidents
plt.figure(figsize=(5,5))
non_death.AC_MASS.value_counts().plot(kind='bar')
plt.title('Distribution of Non-Death Incidents')
plt.xlabel('Aircraft Mass')
plt.ylabel('Number of Incidents')
plt.show()


In [ ]:
plt.figure(figsize=(5,5))
death_incidents.AC_MASS.value_counts().plot(kind='bar')
plt.xlabel('Aircraft Mass')
plt.xticks(rotation=0)
plt.ylabel('Number of Deaths')
plt.title('Number of Deaths from Bird Strike by Aircraft Mass')
plt.show()
plt.figure(figsize=(5,5))
injury_incidents.AC_MASS.value_counts().plot(kind='bar')
plt.xlabel('Aircraft Mass')
plt.xticks(rotation=30)
plt.ylabel('Number of Injuries')
plt.title('Number of Injuries from Bird Strike by Aircraft Mass')
plt.show()

- Is the proportion of incidents that occur at the top 3 airports significantly different from the proportion of flights that occur at the top 3 airports?
  - Result: The proportion of incidents that occur at the top 3 airports is significantly different from the proportion of flights that occur at the top 3 airports at the 0.05 level.
- Is the proportion of incidents that occur during the months 5, 6, 7 at denver airport significantly different from the proportion of flights that occur during the months 5, 6, 7 at denver airport?
  - Result: Significant
- Is the proportion of incidents that occur during each month at the top 3 airports singificantly different from the proportion of flights that occur during each month at the top 3 airports?
 - Result: Significant
- Is the proportion of incidents that occur during Landing at Denver airport significantly different from the proportion of incidents that occur during Landing on average?
  - Result: 
- Is the proportion of incidents that occur during Approach at DFW airport significantly different from the proportion of incidents that occur during Approach on average?
- Is the proportion of incidents that occur during Take-off at Denver airport significantly different from the proportion of incidents that occur during Take-off on average?


- Is the proportion of incidents that occur at low altitude at Denver airport significantly different from the proportion of incidents that occur at low altitude on average?


- Is the proportion of incidents that occur with aircraft between 27,0001 and 272,000 kg at Denver airport significantly different from the proportion of incidents that occur with aircraft between 27,0001 and 272,000 kg on average?

In [ ]:
df.head()

## How has the rate of incidents changed over the past decade?



In [ ]:
flights_per_year_2012_2019 = flights_all_yearly.groupby('INCIDENT_YEAR').Total.sum().tolist()

sns.displot(df, x='INCIDENT_YEAR', kind='hist', aspect=2)

In [ ]:
df['INCIDENT_YEAR'] = df.INCIDENT_DATE.dt.year
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019] 


In [ ]:
incidents_per_year_2012_2019 = df[df['INCIDENT_YEAR'].isin(years)].groupby('INCIDENT_YEAR').INCIDENT.sum().tolist()

In [ ]:
incidents_per_year_2012_2019


In [ ]:
ratio_incidentstoflights = [incidents_per_year_2012_2019[i]/flights_per_year_2012_2019[i] for i in range(len(flights_per_year_2012_2019))]


In [ ]:
plt.plot(years, ratio_incidentstoflights)
plt.xticks(years)
plt.title('Ratio of Incidents to Flights')
plt.xlabel('Year')
plt.ylabel('Ratio of Incidents to Flights')
plt.show()

In [ ]:
# Stat test to determine if the rate of incidents has changed over the past decade
# H0: The rate of incidents has not changed over the past decade
# H1: The rate of incidents has changed over the past decade

# use a chi-square test
# if p-value < 0.05, reject H0

# Chi-Square Assumptions:
# 1. The data is independent
# 2. Expected frequencies are at least 5, but this is not a problem here
# 3. Samples are independent. Trivially true since we are comparing the same sample over time

import statsmodels.stats.proportion as smp

result = smp.proportions_chisquare(incidents_per_year_2012_2019, flights_per_year_2012_2019)
chi2, p, arr = result
print(f'Chi-Square statistic: {chi2}')
print(f'p-value: {p}')

if p < 0.05:
    print('Reject H0')
    print('The rate of incidents has changed over the past decade')
    print('Sample Years:', years)
    
print('Contingeny Table')

print(arr[0])

print('Expected')
print(arr[1])